In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import pandas as pd
import os
from datetime import datetime
from operator import attrgetter

In [2]:
data_folder = r"D:\Documents\trading\converts\data"

In [3]:
info_df = pd.read_csv(os.path.join(data_folder, "converts_info.csv"), index_col='code', encoding='utf-8')
all_data = {code: pd.read_csv(os.path.join(data_folder, "{}.csv".format(code)), parse_dates=[0], index_col=0, encoding='utf-8') for code in info_df.index}

In [4]:
# 双低
for cdf in all_data.values():
    cdf['premium'] = cdf.close_stock / cdf.new_convert_price
    cdf['double_low'] = cdf.close  + cdf.premium

In [5]:
from backtrader.feeds import PandasData
class Addmoredata(PandasData):
    lines = ('open', 'high', 'low', 'close', 'volume', 'money', 'double_low')
    params = (('open', 4), ('high', 5), ('low', 6), ('close', 7), ('volume', 8), ('money', 9), ('double_low', 20))

In [6]:
class ConvertBaseStrategy(bt.Strategy):

    def __init__(self, code2index, max_holding=10):
        self.code2index = code2index
        self.index2code = {v: k for k, v in code2index.items()}
        self.max_holding = max_holding
        self.candidates = []
        self.trade_counter = 0

    def notify_trade(self, trade):
        idx = self.datas.index(trade.data)
        action = 'BOUGHT' if trade.long else 'SELL'
        print(f"{action} {self.index2code[idx]} {abs(trade.size)} @ {trade.price}")
        super(ConvertBaseStrategy, self).notify_trade(trade)

    # def notify_order(self, order):
    #     idx = self.datas.index(order.data)
    #     action = 'BOUGHT' if order.long else 'SELL'
    #     print(f"{action} {self.index2code[idx]} {order.size} @ {order.price}")

    def prenext(self):
        self.next()

    def next(self):
        dt = self.lines.datetime[0]
        # print(self.data.num2date(dt))
        # print(self.positions)
        for idx, data in enumerate(self.datas):
            if data.datetime[0] == dt and data.double_low[0] > 150 and data in self.positions \
                    and self.positions[data].size != 0:
                idx = self.datas.index(data)
                self.trade_counter += 1
                order = self.sell(data, size=self.positions[data].size, exectype=bt.Order.Market,
                                  tradeid=self.trade_counter)  # self.sell(data, self.positions[data].size,
                                                               # exectype=bt.Order.Market)

        vacancy = self.max_holding - len(self.positions)
        for data in self.candidates:
            if vacancy > 0 and data.double_low < 130 and data.close < 110:
                self.trade_counter += 1
                self.buy(data, size=100, exectype=bt.Order.Market, tradeid=self.trade_counter)
                vacancy -= 1

        self.candidates = sorted([data for data in self.datas if dt == data.datetime[0]], key=attrgetter('double_low'))

In [7]:
for idx, (code, df) in enumerate(all_data.items()):
    print(code, df.index.min())
    if idx > 10:
        break

123035 2019-12-09 00:00:00
110033 2016-01-19 00:00:00
110034 2016-01-29 00:00:00
113009 2016-02-04 00:00:00
127003 2016-07-01 00:00:00
128013 2016-08-23 00:00:00
113011 2017-04-05 00:00:00
113012 2017-04-13 00:00:00
128014 2017-05-12 00:00:00
127004 2017-06-26 00:00:00
128015 2017-06-27 00:00:00
113013 2017-07-24 00:00:00


In [9]:
# del cerebro
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000) 
cerebro.broker.setcommission(commission=0.001) 

In [10]:
class AddMoredata(PandasData):
    lines = ('open', 'high', 'low', 'close', 'volume', 'money', 'double_low')
    params = (('open', 4), ('high', 5), ('low', 6), ('close', 7), ('volume', 8), ('money', 9), ('double_low', 20))

In [11]:
code2index = {}
for idx, (code, df) in enumerate(all_data.items()):
    code2index[code] = idx
    feed = AddMoredata(dataname=df, fromdate=datetime(2018, 1, 1), todate=datetime(2021, 8, 26),)
    cerebro.adddata(feed)

In [12]:
cerebro.addstrategy(ConvertBaseStrategy, code2index, 9)


0

In [13]:
# cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
result = cerebro.run()

BOUGHT 128013 100 @ 93.14
BOUGHT 127003 100 @ 93.17
BOUGHT 127004 100 @ 94.05
BOUGHT 128023 100 @ 95.6
BOUGHT 128018 100 @ 95.713
BOUGHT 128015 100 @ 97.1
BOUGHT 113502 100 @ 97.03
BOUGHT 113012 100 @ 97.34
BOUGHT 128025 100 @ 98.86
SELL 127004 100 @ 147.0
SELL 128025 100 @ 150.0


In [98]:
portvalue = cerebro.broker.getvalue()

In [99]:
portvalue

128603.9997

In [ ]:
result[0].b

In [82]:
strat = result[0]
portfolio_stats = strat.analyzers.getbyname('pyfolio')

In [83]:
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()

In [85]:
positions

,Data373,cash
Datetime,,
2018-01-02 00:00:00+00:00,0.0,100000.0
2018-01-03 00:00:00+00:00,0.0,100000.0
2018-01-04 00:00:00+00:00,0.0,100000.0
2018-01-05 00:00:00+00:00,0.0,100000.0
2018-01-08 00:00:00+00:00,0.0,100000.0
...,...,...
2021-08-20 00:00:00+00:00,0.0,100000.0
2021-08-23 00:00:00+00:00,0.0,100000.0
2021-08-24 00:00:00+00:00,0.0,100000.0
